In [1]:
def Calculate_Advection(frame_a,frame_b):
    # High pass the images to make sure the tracking is not of wave velocity itself
    frame_a_hp = frame_a.copy() - smoo(frame_a, 10)
    frame_b_hp = frame_b.copy() - smoo(frame_b, 10)

    winsize = 36 # pixels, interrogation window size in frame A
    searchsize = 36  # pixels, search in image B big enough to contain credible velocity 
    overlap = 18 # pixels, 50% overlap if half of winsize
    dt = 1 # time interval between images, converts pixel displacement to velocity

    # Coordinates of velocity positions in image array
    x, y = pyprocess.get_coordinates(image_size=frame_a.shape, 
                                    search_area_size=searchsize, 
                                    overlap=overlap )

    # Use high-pass images as the PIV source frame_a_hp, frame_b_hp

    u, v, sig2noise = pyprocess.extended_search_area_piv(  frame_a_hp.astype(np.int32), 
                                                        frame_b_hp.astype(np.int32), 
                                                        window_size=winsize, 
                                                        overlap=overlap, 
                                                        dt=dt, 
                                                        search_area_size=searchsize, 
                                                        sig2noise_method='peak2mean')
    # return u/dt, v/dt, sig2noise

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    u, v = filters.replace_outliers( u, v, mask,
                                    method='localmean', 
                                    max_iter=7, 
                                    kernel_size=7)
    
    x, y, u, v = tools.transform_coordinates(x, y, u, v)

    # sum/num will be the average, just remember to avoid zero division
    a2sum = frame_a.copy()*0.0
    a2num = frame_a.copy()*0

    # u and v offsets in nearest pixel, integer
    xi = x.round().astype(int).ravel()
    yi = y.round().astype(int).ravel()
    ui = u.round().astype(int).ravel()
    vi = v.round().astype(int).ravel()


    # Loop over all the little windows in a and add them into a2 
    for idx,vpoint in enumerate(xi.ravel()):
        windowa = (slice(yi[idx]-winsize//2, yi[idx]+winsize//2,1), 
                slice(xi[idx]-winsize//2, xi[idx]+winsize//2,1) ) # source image

        windowa2= (slice(yi[idx]-winsize//2 + -vi[idx], 
                        yi[idx]+winsize//2 + -vi[idx],1), 
                slice(xi[idx]-winsize//2 + ui[idx], 
                        xi[idx]+winsize//2 + ui[idx],1) ) # where to put it
        
        try: 
            a2sum[windowa2] += frame_a[windowa]
            a2num[windowa2] += 1
        except: 
            continue
            
    a2 = (a2sum/(a2num 
                +0.000001)) # avoid division by zero
    return a2

In [81]:
def Show_Advection(frame_a, frame_b):
    
    plt.figure(figsize=[10,5])
    plt.subplot(1, 3, 1)
    plt.imshow(frame_a); plt.title('original frame'); plt.clim(0,200)
    plt.subplot(1, 3, 2)
    plt.imshow(Calculate_Advection(frame_a, frame_b)); plt.title('advected frame'); plt.clim(0,200)
    plt.subplot(1, 3, 3)
    plt.imshow(frame_b); plt.title('original next frame'); plt.clim(0,200)

def Estimate_Advection(frame_a,frame_b):

    plt.figure(figsize=[10,5])
    plt.subplot(1, 2, 1)
    plt.imshow(Calculate_Advection(frame_a, frame_b)); plt.title('advected frame'); plt.clim(0,200); plt.colorbar(shrink=0.5)

    plt.subplot(1, 2, 2)
    plt.imshow(frame_b-Calculate_Advection(frame_a, frame_b),cmap='bwr'); plt.title('Next Frame - Advected Frame'); plt.clim(-50,50); plt.colorbar(shrink=0.5)

In [1]:
# functions for constructing wavenumber and angle, for filtering masks 
def distance_from(a, index):
    i,j = np.indices(a.shape, sparse=True)
    return np.sqrt((i-index[0])**2 + (j-index[1])**2)

def angle_from(a, index):
    i,j = np.indices(a.shape, sparse=True)
    return np.arctan2((j-index[1]),(i-index[0]))

def Show_Brightness(frame_a,frame_b):
    # Fourier transform and shift to center and total wavenumber array kl
    diff = frame_b-frame_a
    diffhat = np.fft.fftshift(np.fft.fft2(diff))

    # Power
    power = np.abs(diffhat)**2

    # Wavenumbers 
    kl = distance_from(diffhat, [diffhat.shape[0]/2, diffhat.shape[1]/2] )

    mask = (  (kl>0) & (kl<=10) ) 

    recon = np.fft.ifft2( np.fft.ifftshift( diffhat*mask ))

    plt.imshow(recon.real, cmap='RdBu_r')
    plt.clim(-20,20); plt.colorbar(shrink=0.7); plt.title('filtered d/dt(brightness)')

def Show_Convergence(frame_a,frame_b,SmoothBorders=False):
    # High pass the images to make sure the tracking is not of wave velocity itself
    frame_a_hp = frame_a.copy() - smoo(frame_a, 10)
    frame_b_hp = frame_b.copy() - smoo(frame_b, 10)

    winsize = 36 # pixels, interrogation window size in frame A
    searchsize = 36  # pixels, search in image B big enough to contain credible velocity 
    overlap = 18 # pixels, 50% overlap if half of winsize
    dt = 1 # time interval between images, converts pixel displacement to velocity

    # Coordinates of velocity positions in image array
    x, y = pyprocess.get_coordinates(image_size=frame_a.shape, 
                                    search_area_size=searchsize, 
                                    overlap=overlap )

    # Use high-pass images as the PIV source frame_a_hp, frame_b_hp

    u, v, sig2noise = pyprocess.extended_search_area_piv(  frame_a_hp.astype(np.int32), 
                                                        frame_b_hp.astype(np.int32), 
                                                        window_size=winsize, 
                                                        overlap=overlap, 
                                                        dt=dt, 
                                                        search_area_size=searchsize, 
                                                        sig2noise_method='peak2mean')

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    u, v = filters.replace_outliers( u, v, mask,
                                    method='localmean', 
                                    max_iter=7, 
                                    kernel_size=7)
    
    x, y, u, v = tools.transform_coordinates(x, y, u, v)

    divx = np.gradient(u)[1]
    divy = -np.gradient(v)[0]
    div = divx+divy
    if SmoothBorders == True:
        window = np.hanning(8)  # 8 pixels wide hanning window (4*2)

        # Apply the window to top and bottom borders
        div[:4, :] = div[:4, :] * window[:4, None]
        div[-4:, :] = div[-4:, :] * window[:4, None]

        # Apply the window to left and right borders
        div[:, :4] = div[:, :4] * window[:4, None].T
        div[:, -4:] = div[:, -4:] * window[:4, None].T

        plt.imshow(smoo(-div, 2), cmap='RdBu_r', clim=[-0.2,0.2]); plt.colorbar(shrink=0.7)
        plt.title('convergence of PIV velocities')  
    else:
        plt.imshow(smoo(-div, 2), cmap='RdBu_r', clim=[-1,1]); plt.colorbar(shrink=0.7)
        plt.title('convergence of PIV velocities')    

def Show_Contour(frame_a,frame_b,SmoothBorders=False):
    # High pass the images to make sure the tracking is not of wave velocity itself
    frame_a_hp = frame_a.copy() - smoo(frame_a, 10)
    frame_b_hp = frame_b.copy() - smoo(frame_b, 10)

    winsize = 36 # pixels, interrogation window size in frame A
    searchsize = 36  # pixels, search in image B big enough to contain credible velocity 
    overlap = 18 # pixels, 50% overlap if half of winsize
    dt = 1 # time interval between images, converts pixel displacement to velocity

    # Use high-pass images as the PIV source frame_a_hp, frame_b_hp

    u, v, sig2noise = pyprocess.extended_search_area_piv(  frame_a_hp.astype(np.int32), 
                                                        frame_b_hp.astype(np.int32), 
                                                        window_size=winsize, 
                                                        overlap=overlap, 
                                                        dt=dt, 
                                                        search_area_size=searchsize, 
                                                        sig2noise_method='peak2mean')
    # Coordinates of velocity positions in image array
    x, y = pyprocess.get_coordinates(image_size=frame_a.shape, 
                                    search_area_size=searchsize, 
                                    overlap=overlap )

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    u, v = filters.replace_outliers( u, v, mask,
                                    method='localmean', 
                                    max_iter=7, 
                                    kernel_size=7)
    
    x, y, u, v = tools.transform_coordinates(x, y, u, v)

    divx = np.gradient(u)[1]
    divy = -np.gradient(v)[0]
    div = divx+divy
    if SmoothBorders == True:
        # transform the first 4 rows and columuns into NaN
        window = np.hanning(8)  # 8 pixels wide hanning window (4*2)

        # Apply the window to top and bottom borders
        div[:4, :] = div[:4, :] * window[:4, None]
        div[-4:, :] = div[-4:, :] * window[:4, None]

        # Apply the window to left and right borders
        div[:, :4] = div[:, :4] * window[:4, None].T
        div[:, -4:] = div[:, -4:] * window[:4, None].T


    # Fourier transform and shift to center and total wavenumber array kl
    diff = frame_b-frame_a
    diffhat = np.fft.fftshift(np.fft.fft2(diff))

    # Wavenumbers 
    kl = distance_from(diffhat, [diffhat.shape[0]/2, diffhat.shape[1]/2] )

    mask = (  (kl>0) & (kl<=10) ) 

    recon = np.fft.ifft2( np.fft.ifftshift( diffhat*mask ))

    plt.pcolormesh(recon.real, cmap='RdBu_r')
    plt.clim(-20,20); plt.title('d/dt(brightness) and conv(PIV wind)')

    plt.contour(x,y, np.flipud(smoo(-div, 2)), levels=(-5+np.arange(11))/20., cmap='RdBu_r')
    if SmoothBorders == False:
        plt.ylim(950,50); plt.xlim(500,1400)  
    

In [ ]:
def Get_Matrices(frame_a,frame_b):
    # High pass the images to make sure the tracking is not of wave velocity itself
    frame_a_hp = frame_a.copy() - smoo(frame_a, 10)
    frame_b_hp = frame_b.copy() - smoo(frame_b, 10)

    winsize = 36 # pixels, interrogation window size in frame A
    searchsize = 36  # pixels, search in image B big enough to contain credible velocity 
    overlap = 18 # pixels, 50% overlap if half of winsize
    dt = 1 # time interval between images, converts pixel displacement to velocity

    # Use high-pass images as the PIV source frame_a_hp, frame_b_hp

    u, v, sig2noise = pyprocess.extended_search_area_piv(  frame_a_hp.astype(np.int32), 
                                                        frame_b_hp.astype(np.int32), 
                                                        window_size=winsize, 
                                                        overlap=overlap, 
                                                        dt=dt, 
                                                        search_area_size=searchsize, 
                                                        sig2noise_method='peak2mean')
    # Coordinates of velocity positions in image array
    x, y = pyprocess.get_coordinates(image_size=frame_a.shape, 
                                    search_area_size=searchsize, 
                                    overlap=overlap )

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    mask = validation.global_std(u, v, std_threshold=3)
    #replace outliers with NaNs
    u[mask] = np.nan
    v[mask] = np.nan

    u, v = filters.replace_outliers( u, v, mask,
                                    method='localmean', 
                                    max_iter=7, 
                                    kernel_size=7)
    
    x, y, u, v = tools.transform_coordinates(x, y, u, v)

    divx = np.gradient(u)[1]
    divy = -np.gradient(v)[0]
    div = divx+divy

    # Fourier transform and shift to center and total wavenumber array kl
    diff = frame_b-frame_a
    diffhat = np.fft.fftshift(np.fft.fft2(diff))

    # Wavenumbers 
    kl = distance_from(diffhat, [diffhat.shape[0]/2, diffhat.shape[1]/2] )

    mask = (  (kl>0) & (kl<=10) ) 

    recon = np.fft.ifft2( np.fft.ifftshift( diffhat*mask ))

    return smoo(-div, 2), recon.real

def resize_matrix(original, target_dimensions):
    original, target = Get_Matrices(frame_a, frame_b)

    # Determine the current dimensions of the original matrix
    original_dimensions = original.shape
    target_dimensions = target.shape

    # Step 1: Define the original grid
    x = np.linspace(0, 1, original_dimensions[1])  
    y = np.linspace(0, 1, original_dimensions[0])  

    # Step 2: Define the function on this grid
    X, Y = np.meshgrid(x, y, indexing='ij')
    Z = original.T

    # Step 3: Create a finer grid for interpolation
    x_fine = np.linspace(0, 1, target_dimensions[1])  
    y_fine = np.linspace(0, 1, target_dimensions[0])  

    # Step 4: Use RegularGridInterpolator
    interpolator = RegularGridInterpolator((x, y), Z)
    X_fine, Y_fine = np.meshgrid(x_fine, y_fine, indexing='ij')
    points_fine = np.array([X_fine.ravel(), Y_fine.ravel()]).T
    Z_fine = interpolator(points_fine).reshape(target_dimensions[1], target_dimensions[0])

    return Z_fine.T


In [2]:
def Show_Candidates(frame_a,frame_b,plot_option='no'):
    con, bri = Get_Matrices(frame_a,frame_b)
    con=resize_matrix(con, bri.shape)

    # create a tuple array, that takes for each cell the con and bri values
    candidates = np.zeros((con.shape[0], con.shape[1]), dtype=[('con', float), ('bri', float)])
    candidates['con'] = con
    candidates['bri'] = bri

    # Define a threshold for closeness to the 1:1 line
    threshold = 0.5  

    # Get the points close to the 1:1 line
    close_to_line = np.abs(candidates['bri'] - candidates['con']) < threshold

    if plot_option == 'yes':
        # Scatter plot of con vs bri with the required axis settings and aspect ratio
        plt.figure(figsize=(10, 5))  

        #plot the close to line in red and the rest in blue
        plt.scatter(candidates['bri'][close_to_line], candidates['con'][close_to_line], s=1, color='red')
        plt.scatter(candidates['bri'][~close_to_line], candidates['con'][~close_to_line], s=1, color='blue')
        plt.xlabel('Brightness')
        plt.ylabel('Convergence')

        # Adjust axes to be centered on (0, 0)
        plt.axhline(0, color='black', linewidth=0.8)
        plt.axvline(0, color='black', linewidth=0.8)

        # Set the axis limits 
        plt.xlim(-10, 10)
        plt.ylim(-5, 5)

        # Plot a line with a slope of 1:1 for convergence:brightness
        plt.plot([bri.min(), bri.max()], [bri.min(), bri.max()], 'k--', lw=2)  # Plot a 1:1 line for reference

        plt.title('Scatter Plot of Convergence vs. Brightness')
        plt.show()

    # Retrieve the coordinates of the points close to the 1:1 line
    close_points_indices = np.nonzero(close_to_line)
    close_points_coordinates = list(zip(close_points_indices[0], close_points_indices[1]))

    # plot frame_b with the close points in red
    plt.figure(figsize=(10, 5))
    plt.imshow(frame_b, cmap='gray')
    plt.scatter([p[1] for p in close_points_coordinates], [p[0] for p in close_points_coordinates], s=1, color='red')
    plt.title('Potential candidates for GW')
    plt.show()

In [ ]:
def save_frames_from_video(video_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f'frame{count}.png')
        cv2.imwrite(frame_path, frame)
        count += 1

    cap.release()
    cv2.destroyAllWindows()

def crop_to_central_data(image_path,number):
    """
    Crop the image to the central data area based on hardcoded coordinates.

    :param image_path: Path to the image file.
    :return: Cropped Image object.
    """
    crop_coordinates = (125, 120, 900, 890) 
    with Image.open(image_path) as img:
        # Crop the image to the predefined coordinates
        # Extract the directory and base name without extension
        directory = os.path.dirname(image_path)
        base_name = os.path.basename(image_path)
        name, extension = os.path.splitext(base_name)

        # Create the new file name and save the cropped image
        new_file_name = f"{'image_'}{number}{extension}"
        cropped_img = img.crop(crop_coordinates)
        cropped_img.save(os.path.join(directory, new_file_name))

# Example usage:
#cropped_image = crop_to_central_data(path_to_image,0)
# The function calls are commented out to prevent execution in the PCI.

In [ ]:
def Fourrier_Analysis(frame):
    # FFT2 to identify horizontal wavenumber vector 
    # ChatGPT wrote this code and I tested/adapted from there 

    # remove enough colmuns or rows to make it square
    if frame.shape[0] > frame.shape[1]:
        frame = frame[:frame.shape[1],:]
    elif frame.shape[0] < frame.shape[1]:
        frame = frame[:,:frame.shape[0]]

    cloud_pattern = frame
    

    # Generate a sample image (replace this with your cloud probability distribution)
    image_size = cloud_pattern.shape[0]

    # Apply Fourier Transform
    fft_result = fft2(cloud_pattern)

    # Shift zero frequency components to the center
    fft_result_shifted = fftshift(fft_result)

    # Calculate amplitude
    amplitude = np.abs(fft_result_shifted)


    # Create 2D wavenumber array
    kx = np.fft.fftshift(np.fft.fftfreq(image_size, d=1/image_size)) 
    ky = np.fft.fftshift(np.fft.fftfreq(image_size, d=1/image_size))
    kx, ky = np.meshgrid(kx, ky)

    # Calculate total wavenumber array
    wavenumbers = np.sqrt(kx**2 + ky**2)


    # Calculate the polar coordinates
    radius = np.sqrt(kx**2 + ky**2)
    theta = np.arctan2(ky, kx)

    # Average the amplitude where the TOTAL wavenumber is between 4 and 20
    min_wavenumber = 2
    max_wavenumber = 20
    in_k_range = (radius>min_wavenumber) & (radius<max_wavenumber)


    # Create theta bins
    num_bins = 36  # 10 degrees each bin 
    theta_bins = np.linspace(np.min(theta), np.max(theta), num_bins + 1)

    # Calculate the mean amplitude in each theta bin
    mean_amplitude = np.zeros(num_bins)
    for i in range(num_bins):
        in_bin = (theta >= theta_bins[i]) & (theta < theta_bins[i + 1])
        mean_amplitude[i] = np.mean(amplitude * in_k_range * in_bin)


    # -------- chatGPT fits it, although really just argmax and max-mean suffices 
    # Define the bisinusoidal fit function
    def bisinusoidal_fit(theta, amplitude, phase, offset):
        return amplitude * np.sin(2*theta + phase) + offset
    # Define the sinusoidal fit function
    def sinusoidal_fit(theta, amplitude, phase, offset):
        return amplitude * np.sin(theta + phase) + offset

    # Initial guess for the fit parameters
    initial_guess = [1.0, 0.0, 0.0]

    # Fit the sinusoidal curve to the azimuthal variation
    popt, _ = curve_fit(bisinusoidal_fit, theta_bins[:-1], mean_amplitude, p0=initial_guess)
    # ------------


    # Plot the original image, amplitude, the annulus, and the fitted sinusoidal curve
    plt.figure(figsize=(16, 4))

    plt.subplot(141)
    plt.pcolormesh(cloud_pattern, cmap='viridis')
    plt.title('Original Image')

    plt.subplot(142)
    plt.pcolormesh(np.log1p(amplitude), cmap='viridis')
    plt.title('Log Amplitude Spectrum')

    plt.subplot(143)
    plt.pcolormesh(kx,ky, np.log1p(amplitude), cmap='viridis')
    ax = plt.gca()
    ax.set_xlim([-30, 30])
    ax.set_ylim([-30, 30])
    plt.contour(kx,ky,radius, levels=[min_wavenumber, max_wavenumber], colors='r', linewidths=2)
    plt.title('Annulus (Wavenumbers 4-20)')

    plt.subplot(144)
    plt.plot(theta_bins[:-1] *180/3.1415 + 360/num_bins/2, mean_amplitude)
    plt.title('amplitude vs. angle in 4-20 wavenumber band')
    plt.plot(theta_bins[:-1] *180/3.1415 + 360/num_bins/2, sinusoidal_fit(2*theta_bins[:-1], *popt), 'r-', label='Sinusoidal Fit')

    plt.plot() 

    plt.tight_layout()
    plt.show()


    argmx = np.argmax(mean_amplitude)
    maxangle = theta_bins[ argmx ]
    maxangledeg = theta_bins[ argmx ] *180/3.1415 + 360/num_bins/2

    print('angle is ', argmx, maxangle, maxangledeg, maxangledeg+180)
    print('strength is ',np.max(mean_amplitude)/np.mean(mean_amplitude) )

    # Print the fitted parameters
    #amplitude_fit, phase_fit, offset_fit = popt
    #print(f"Amplitude of the Fit: {amplitude_fit}")
    #print(f"Phase of the Fit: {phase_fit*1800./3.142}")
    #print(f"Offset of the Fit: {offset_fit}")
